In [ ]:
import pandas as pd
import numpy as np

awards_players = pd.read_csv("../data/awards_players.csv")
coaches = pd.read_csv("../data/coaches.csv")
players_teams = pd.read_csv("../data/players_teams.csv")
players = pd.read_csv("../data/players.csv")
series_post = pd.read_csv("../data/series_post.csv")
teams_post = pd.read_csv("../data/teams_post.csv")
teams = pd.read_csv("../data/teams.csv")


In [ ]:
awards_players = awards_players.loc[:, awards_players.nunique() > 1]
coaches = coaches.loc[:, coaches.nunique() > 1]
players_teams = players_teams.loc[:, players_teams.nunique() > 1]
players = players.loc[:, players.nunique() > 1]
series_post = series_post.loc[:, series_post.nunique() > 1]
teams_post = teams_post.loc[:, teams_post.nunique() > 1]
teams = teams.loc[:, teams.nunique() > 1]

In [ ]:
teams = teams.drop(columns =['name', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'arena'], errors='ignore')
teams["perW"] = teams["won"] / teams["GP"] # Winning percentage
teams["perL"] = teams["lost"] / teams["GP"] # Losing percentage
teams["o_fgEf"] = teams["o_fgm"] / teams["o_fga"] # Field goals  Percentage
teams["o_ftEf"] = teams["o_ftm"] / teams["o_fta"] # Free throws Percentage
teams["o_3pEf"] = teams["o_3pm"] / teams["o_3pa"] # Three-pointers Percentage
teams["o_per"] = teams["o_pts"] + teams["o_pts"]*1.5 + teams["o_oreb"]*1.2 - teams["o_to"]* 1.5 - teams["o_pf"] # Performance
teams["d_fgEf"] = teams["d_fgm"] / teams["d_fga"] # Field goals by opponents Percentage
teams["d_ftEf"] = teams["d_ftm"] / teams["d_fta"] # Free throws by opponents Percentage
teams["d_3pEf"] = teams["d_3pm"] / teams["d_3pa"] # Three-pointers by opponents Percentage
teams["pts_diff"] =  teams["o_pts"] -  teams["d_pts"] # Difference between points scored and conceded
teams["pts_mpg"] =  teams["o_pts"] / teams["GP"] # Average points scored per game
teams["pts_tmp"] =  teams["d_pts"] / teams["GP"] # Average points conceded per game
teams["effec_fg"] =  (teams["pts_tmp"] + 0.5 * teams["o_3pm"]) /  teams["o_fga"] # Effective Field Goal Percentage
#Possessions 
#teams["possM"] = 0.5 * ((teams["o_fga"] + 0.4 * teams["o_fta"] - 1.07 * (teams["o_oreb"] / (teams["o_oreb"] + teams["d_dreb"])) * (teams["o_fga"] - teams["o_fgm"]) + teams["o_to"]) + 
#                       (teams["d_fga"] + 0.4 * teams["d_fga"] - 1.07 * (teams["d_oreb"] / (teams["d_oreb"] + teams["o_dreb"])) * (teams["d_fga"] - teams["d_fgm"]) + teams["d_to"]))

teams["team_poss"] = 0.96*(teams["o_fga"] + teams["o_to"]+0.44*teams["o_fta"] - teams["o_oreb"])
#teams["team_poss"] = (teams["o_fga"] + 0.4 * teams["o_fta"] - 1.07 * (teams["o_oreb"] / (teams["o_oreb"] + teams["d_dreb"])) * (teams["o_fga"] - teams["o_fgm"]) + teams["o_to"])

teams["opp_poss"] = 0.96*(teams["d_fga"] + teams["d_to"]+0.44*teams["d_fta"] - teams["d_oreb"])
#teams["opp_poss"] = (teams["d_fga"] + 0.4 * teams["d_fga"] - 1.07 * (teams["d_oreb"] / (teams["d_oreb"] + teams["o_dreb"])) * (teams["d_fga"] - teams["d_fgm"]) + teams["d_to"])

teams["off_rtg"] = (teams["o_pts"] / teams["team_poss"]) * 100 # Offensive Rating
teams["def_rtg"] = (teams["d_pts"] / teams["team_poss"]) * 100 # Defensive Rating
teams["minutes_team"] = teams['min'] / 5

teams["pace"] = 40 * ((teams["team_poss"] + teams["opp_poss"] ) / (2 * (teams["minutes_team"])))


for year, year_data in teams.groupby("year"):
    lg_pace = teams[teams['year'] == year]['pace'].mean()
    teams.loc[teams['year'] == year,'lg_pace'] = lg_pace

teams.to_csv('new_teams.csv', index=False)


In [ ]:
#for year, year_data in players_teams.groupby("year"):
#    lg_ast = year_data['assists'].sum()             #+ year_data['PostAssists'].sum()
#    lg_fg = year_data['fgMade'].sum()               #+ year_data['PostfgMade'].sum()
#    lg_ft = year_data['ftMade'].sum()               #+ year_data['PostftMade'].sum()
#    lg_fga = year_data['fgAttempted'].sum()         #+ year_data['PostfgAttempted'].sum()
#    lg_fta = year_data['ftAttempted'].sum()         #+ year_data['PostftAttempted'].sum()
#    lg_trb = year_data['rebounds'].sum()            #+ year_data['PostRebounds'].sum()
#    lg_orb = year_data['oRebounds'].sum()           #+ year_data['PostoRebounds'].sum()
#    lg_drb = year_data['dRebounds'].sum()           #+ year_data['PostdRebounds'].sum()
#    lg_to = year_data['turnovers'].sum()            #+ year_data['PostTurnovers'].sum()
#    lg_pf = year_data['PF'].sum()                   #+ year_data['PostPF'].sum()
#    lg_pts = year_data['points'].sum()              #+ year_data['PostPoints'].sum()
#    print(year,lg_ast, lg_pts)
#
#
#print("\n")
#for year, year_data in teams.groupby("year"):
#    lg_pts = year_data['o_pts'].sum()             
#    lg_ast = year_data['o_asts'].sum()              
#    print(year, lg_ast, lg_pts)


In [ ]:
for year, year_data in players_teams.groupby("year"):
    lg_ast = year_data['assists'].sum()             + year_data['PostAssists'].sum()
    lg_fg = year_data['fgMade'].sum()               + year_data['PostfgMade'].sum()
    lg_ft = year_data['ftMade'].sum()               + year_data['PostftMade'].sum()
    lg_fga = year_data['fgAttempted'].sum()         + year_data['PostfgAttempted'].sum()
    lg_fta = year_data['ftAttempted'].sum()         + year_data['PostftAttempted'].sum()
    lg_trb = year_data['rebounds'].sum()            + year_data['PostRebounds'].sum()
    lg_orb = year_data['oRebounds'].sum()           + year_data['PostoRebounds'].sum()
    lg_drb = year_data['dRebounds'].sum()           + year_data['PostdRebounds'].sum()
    lg_to = year_data['turnovers'].sum()            + year_data['PostTurnovers'].sum()
    lg_pf = year_data['PF'].sum()                   + year_data['PostPF'].sum()
    lg_pts = year_data['points'].sum()              + year_data['PostPoints'].sum()
    
    factor = (2 / 3) - (0.5 * (lg_ast / lg_fg)) / (2 * (lg_fg / lg_ft))
    vop    = lg_pts / (lg_fga - lg_orb + lg_to + 0.44 * lg_fta)
    drbP  = lg_drb /lg_trb
    
    for tmID, tm_data in year_data.groupby('tmID'):
        tm_ast = tm_data['assists'].sum()       + tm_data['PostAssists'].sum()
        tm_fg = tm_data['fgMade'].sum()         + tm_data['PostfgMade'].sum()
        tm_pts = tm_data['points'].sum()        + year_data['PostPoints'].sum()
  
        for player, player_data in tm_data.groupby('playerID'):
            player_min = player_data['minutes'].sum() + player_data['PostMinutes'].sum()
            player_3p = player_data['threeMade'].sum() + player_data['PostthreeMade'].sum()
            player_ast = player_data['assists'].sum() + player_data['PostAssists'].sum()
            player_fg = player_data['fgMade'].sum() + player_data['PostfgMade'].sum()
            player_ft = player_data['ftMade'].sum() + player_data['PostftMade'].sum()
            player_fga = player_data['fgAttempted'].sum() + player_data['PostfgAttempted'].sum()
            player_fta = player_data['ftAttempted'].sum() + player_data['PostftAttempted'].sum()
            player_to = player_data['turnovers'].sum() + player_data['PostTurnovers'].sum()
            player_trb = player_data['rebounds'].sum() + player_data['PostRebounds'].sum()
            player_orb = player_data['oRebounds'].sum() + player_data['PostoRebounds'].sum()
            player_drb = player_data['dRebounds'].sum() + player_data['PostdRebounds'].sum()
            player_stl = player_data['steals'].sum() + player_data['PostSteals'].sum()
            player_blk = player_data['blocks'].sum() + player_data['PostBlocks'].sum()
            player_pf = player_data['PF'].sum() + player_data['PostPF'].sum()

            if(player_min == 0):
                uper = 0
            else:
                uper = (1 / player_min) * ( player_3p 
                    + (2/3) * player_ast
                    + (2 - factor * (tm_ast / tm_fg)) * player_fg
                    + (player_ft * 0.5 * (2 - (1/3) * (tm_ast / tm_fg)))
                    - vop * player_to
                    - vop * drbP * (player_fga - player_fg)
                    - vop * 0.44 * (0.44 + (0.56 * drbP)) * (player_fta - player_ft)
                    + vop * (1 -drbP) * (player_trb - player_orb)
                    + vop * drbP * player_orb
                    + vop * player_stl
                    + vop * drbP * player_blk
                    - player_pf * ((lg_ft / lg_pf) - 0.44 * (lg_fta / lg_pf) * vop)
                )

            players_teams.loc[
                (players_teams['year'] == year) &
                (players_teams['playerID'] == player) &
                (players_teams['tmID'] == tmID),
                'uper'] = uper
            
    lg_uper = players_teams[players_teams['year'] == year]['uper'].mean()
    players_teams.loc[players_teams['year'] == year,'lg_uper'] = lg_uper



               

14. o_fgm: Field goals made
15. o_fga: Field goals attempted
16. o_ftm: Free throws made
17. o_fta: Free throws attempted
18. o_3pm: Three-pointers made
19. o_3pa: Three-pointers attempted
20. o_oreb: Offensive rebounds
21. o_dreb: Defensive rebounds
22. o_reb: Total rebounds
23. o_asts: Assists
24. o_pf: Personal fouls
25. o_stl: Steals
26. o_to: Turnovers
27. o_blk: Blocks
28. o_pts: Points scored
29. d_fgm: Field goals made by opponents
30. d_fga: Field goals attempted by opponents
31. d_ftm: Free throws made by opponents
32. d_fta: Free throws attempted by opponents
33. d_3pm: Three-pointers made by opponents
34. d_3pa: Three-pointers attempted by opponents
35. d_oreb: Offensive rebounds by opponents
36. d_dreb: Defensive rebounds by opponents
37. d_reb: Total rebounds by opponents
38. d_asts: Assists by opponents
39. d_pf: Personal fouls by opponents
40. d_stl: Steals by opponents
41. d_to: Turnovers by opponents
42. d_blk: Blocks by opponents
43. d_pts: Points scored by opponents

44. tmORB: Team offensive rebounds
45. tmDRB: Team defensive rebounds
46. tmTRB: Team total rebounds
Opponent Team Rebounding:
47. opptmORB: Opponent team offensive rebounds
48. opptmDRB: Opponent team defensive rebounds
49. opptmTRB: Opponent team total rebounds



Performance Ofensiva=o_pts+(o_asts×1.5)+(o_oreb×1.2)−(o_to×1.5)−o_pf

 (total de pontos feitos + rebotes totais + tocos
+ bolas recuperados + assistências) - (total de
arremessos errados + erros + faltas cometidas).


Poss - Possessions (available since the 1973-74 season in the NBA); the formula for teams is 0.5 * ((Tm FGA + 0.4 * Tm FTA - 1.07 * (Tm ORB / (Tm ORB + Opp DRB)) * (Tm FGA - Tm FG) + Tm TOV) + (Opp FGA + 0.4 * Opp FTA - 1.07 * (Opp ORB / (Opp ORB + Tm DRB)) * (Opp FGA - Opp FG) + Opp TOV)). This formula estimates possessions based on both the team's statistics and their opponent's statistics, then averages them to provide a more stable estimate. Please see the article Calculating Individual Offensive and Defensive Ratings for more information.

In [ ]:
mPT = pd.merge(players_teams, teams, on=['year', 'tmID'], how='left')
#
#
#mPT["qAST"] = ((mPT["minutes"] / (mPT["minutes_team"] / 5)) * (1.14 * ((mPT["o_asts"] - mPT["assists"]) / mPT["o_fgm"]))) + ((((mPT["o_asts"] / mPT["minutes_team"]) * mPT["minutes"] * 5 - mPT["assists"]) / ((mPT["o_fgm"] 
#                              / mPT["minutes_team"]) * mPT["minutes"] * 5 - mPT["fgMade"])) * (1 - (mPT["minutes"] / (mPT["minutes_team"] / 5))))
#
#
#mPT["FG_Part"] = mPT["fgMade"] * (1 - 0.5 * ((mPT["points"] - mPT["ftMade"]) / (2 * mPT["fgAttempted"])) * mPT["qAST"])
#    
#mPT["ASP_Part"] = 0.5 * (((mPT["o_pts"] - mPT["o_ftm"]) - (mPT["points"] - mPT["ftMade"])) / (2 * (mPT["o_fga"] - mPT["fgAttempted"]))) * mPT["assists"]
#
#mPT["FT_Part"] = (1 - (1 - (mPT["ftMade"] / mPT["ftAttempted"])) ** 2) * 0.4 * mPT["ftAttempted"]
#
#mPT["Team_Scoring_Poss"] = mPT["o_fgm"] + (1 - (1 - (mPT["o_ftm"] / mPT["o_fta"]))**2) * mPT["fgAttempted"] * 0.4
#
#mPT["Team_ORB%"] = mPT["o_oreb"] / (mPT["o_oreb"] + (mPT["d_reb"] - mPT["d_oreb"]))
#
#mPT["Team_Play%"] = mPT["Team_Scoring_Poss"] / (mPT["o_fga"] + mPT["o_fta"] * 0.4 + mPT["o_to"])
#
#mPT["Team_ORB_Weight"] = ((1 - mPT["Team_ORB%"]) * mPT["Team_Play%"]) / ((1 - mPT["Team_ORB%"]) * mPT["Team_Play%"] + mPT["Team_ORB%"] * (1 - mPT["Team_Play%"]))
#
#mPT["ORB_Part"] = mPT["oRebounds"] * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]
#
## Scoring Possesssions
#mPT["ScPoss"] = (mPT["FG_Part"] + mPT["ASP_Part"] + mPT["FT_Part"]) * (1 - (mPT["o_oreb"] / mPT["Team_Scoring_Poss"]) * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]) + mPT["ORB_Part"]
#
## Missed FG Possessions
#mPT["FGxPoss"] = (mPT["fgAttempted"] - mPT["fgMade"]) * (1 - 1.07 * mPT["Team_ORB%"])
#
## Missed FT Possessions
#mPT["FTxPoss"] = ((1 - (mPT["ftMade"] / mPT["ftAttempted"])) ** 2) * 0.4 * mPT["ftAttempted"]
#
##Total Possessions
#mPT["TotPoss"] = mPT["ScPoss"] + mPT["FGxPoss"] + mPT["FTxPoss"] + mPT["turnovers"]
#
#mPT["PProd_FG_Part"] = 2 * (mPT["fgMade"] + 0.5 * mPT["threeMade"]) * (1 - 0.5 * ((mPT["points"] - mPT["fgMade"]) / (2 * mPT["fgMade"])) * mPT["qAST"])
#
#mPT["PProd_AST_Part"] = 2 * ((mPT["o_fgm"] - mPT["fgMade"] + 0.5 * (mPT["o_3pa"] - mPT["threeMade"])) / (mPT["o_fgm"] - mPT["fgMade"])) * 0.5 * (((mPT["o_pts"] - mPT["o_ftm"]) - (mPT["points"] - mPT["ftMade"])) / (2 * (mPT["o_fga"] - mPT["fgAttempted"]))) * mPT["assists"] 
#
#mPT["PProd_ORB_Part"] = mPT["oRebounds"] * mPT["Team_ORB_Weight"] * mPT["Team_Play%"] * (mPT["o_pts"] / ( mPT["o_fgm"] + (1 - (1 - (mPT["o_ftm"] / mPT["o_fta"])) ** 2) * 0.4 * mPT["o_fta"]))
#
## Indivisual Points Produced
#mPT["PProd"] = (mPT["PProd_FG_Part"] + mPT["PProd_AST_Part"] + mPT["ftMade"]) * (1 - (mPT["o_oreb"] / mPT["Team_Scoring_Poss"]) * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]) + mPT["PProd_ORB_Part"]
#
## Individual Offensive Rating
#mPT["ORtg"] = 100 * (mPT["PProd"] / mPT["TotPoss"])
#
## Floor Percentage
#mPT["Floor%"] = mPT["ScPoss"] / mPT["TotPoss"]


#Modelos como regressão logística, árvores de decisão ou redes neurais 


# Cálculos com verificações de divisão por zero
mPT["qAST"] = ((mPT["minutes"] / np.where(mPT["minutes_team"] != 0, (mPT["minutes_team"] / 5), 1)) * 
               (1.14 * np.where(mPT["o_fgm"] != 0, (mPT["o_asts"] - mPT["assists"]) / mPT["o_fgm"], 0)) + 
               (((mPT["o_asts"] / np.where(mPT["minutes_team"] != 0, mPT["minutes_team"], 1)) * mPT["minutes"] * 5 - mPT["assists"]) / 
                (np.where(mPT["o_fgm"] != 0, (mPT["o_fgm"] / mPT["minutes_team"]) * mPT["minutes"] * 5 - mPT["fgMade"], 1)) * 
                (1 - (mPT["minutes"] / np.where(mPT["minutes_team"] != 0, mPT["minutes_team"] / 5, 1)))))

mPT["FG_Part"] = mPT["fgMade"] * (1 - 0.5 * ((mPT["points"] - mPT["ftMade"]) / (2 * mPT["fgAttempted"])) * mPT["qAST"])

mPT["ASP_Part"] = 0.5 * (((mPT["o_pts"] - mPT["o_ftm"]) - (mPT["points"] - mPT["ftMade"])) / (2 * (mPT["o_fga"] - mPT["fgAttempted"]))) * mPT["assists"]

mPT["FT_Part"] = (1 - (1 - np.where(mPT["ftAttempted"] != 0, mPT["ftMade"] / mPT["ftAttempted"], 0)) ** 2) * 0.4 * mPT["ftAttempted"]

mPT["Team_Scoring_Poss"] = mPT["o_fgm"] + (1 - (1 - np.where(mPT["o_fta"] != 0, mPT["o_ftm"] / mPT["o_fta"], 0))**2) * mPT["fgAttempted"] * 0.4

mPT["Team_ORB%"] = np.where((mPT["o_oreb"] + (mPT["d_reb"] - mPT["d_oreb"])) != 0, mPT["o_oreb"] / (mPT["o_oreb"] + (mPT["d_reb"] - mPT["d_oreb"])), 0)

mPT["Team_Play%"] = np.where((mPT["o_fga"] + mPT["o_fta"] * 0.4 + mPT["o_to"]) != 0, mPT["Team_Scoring_Poss"] / (mPT["o_fga"] + mPT["o_fta"] * 0.4 + mPT["o_to"]), 0)

mPT["Team_ORB_Weight"] = np.where(((1 - mPT["Team_ORB%"]) * mPT["Team_Play%"] + mPT["Team_ORB%"] * (1 - mPT["Team_Play%"])) != 0,
                                  ((1 - mPT["Team_ORB%"]) * mPT["Team_Play%"]) / ((1 - mPT["Team_ORB%"]) * mPT["Team_Play%"] + mPT["Team_ORB%"] * (1 - mPT["Team_Play%"])), 0)

mPT["ORB_Part"] = mPT["oRebounds"] * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]

# Scoring Possessions
mPT["ScPoss"] = (mPT["FG_Part"] + mPT["ASP_Part"] + mPT["FT_Part"]) * (1 - np.where(mPT["Team_Scoring_Poss"] != 0, (mPT["o_oreb"] / mPT["Team_Scoring_Poss"]), 0) * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]) + mPT["ORB_Part"]

# Missed FG Possessions
mPT["FGxPoss"] = (mPT["fgAttempted"] - mPT["fgMade"]) * (1 - 1.07 * mPT["Team_ORB%"])

# Missed FT Possessions
mPT["FTxPoss"] = ((1 - np.where(mPT["ftAttempted"] != 0, mPT["ftMade"] / mPT["ftAttempted"], 0)) ** 2) * 0.4 * mPT["ftAttempted"]

# Total Possessions
mPT["TotPoss"] = mPT["ScPoss"] + mPT["FGxPoss"] + mPT["FTxPoss"] + mPT["turnovers"]

mPT["PProd_FG_Part"] = 2 * (mPT["fgMade"] + 0.5 * mPT["threeMade"]) * (1 - 0.5 * np.where((2 * mPT["fgMade"]) != 0, ((mPT["points"] - mPT["fgMade"]) / (2 * mPT["fgMade"])), 0) * mPT["qAST"])

mPT["PProd_AST_Part"] = 2 * np.where((mPT["o_fgm"] - mPT["fgMade"]) != 0, 
                                     ((mPT["o_fgm"] - mPT["fgMade"] + 0.5 * (mPT["o_3pa"] - mPT["threeMade"])) / (mPT["o_fgm"] - mPT["fgMade"])) * 0.5 * 
                                     ((mPT["o_pts"] - mPT["o_ftm"] - (mPT["points"] - mPT["ftMade"])) / (2 * (mPT["o_fga"] - mPT["fgAttempted"]))) * mPT["assists"], 0)

mPT["PProd_ORB_Part"] = mPT["oRebounds"] * mPT["Team_ORB_Weight"] * mPT["Team_Play%"] * np.where(mPT["o_fgm"] != 0, (mPT["o_pts"] / (mPT["o_fgm"] + (1 - (1 - np.where(mPT["o_fta"] != 0, mPT["o_ftm"] / mPT["o_fta"], 0)) ** 2) * 0.4 * mPT["o_fta"])), 0)

# Individual Points Produced
mPT["PProd"] = (mPT["PProd_FG_Part"] + mPT["PProd_AST_Part"] + mPT["ftMade"]) * (1 - np.where(mPT["Team_Scoring_Poss"] != 0, (mPT["o_oreb"] / mPT["Team_Scoring_Poss"]), 0) * mPT["Team_ORB_Weight"] * mPT["Team_Play%"]) + mPT["PProd_ORB_Part"]

# Individual Offensive Rating
mPT["ORtg"] = 100 * np.where(mPT["TotPoss"] != 0, mPT["PProd"] / mPT["TotPoss"], 0)

# Floor Percentage
mPT["Floor%"] = np.where(mPT["TotPoss"] != 0, mPT["ScPoss"] / mPT["TotPoss"], 0)

mPT = mPT.drop(columns =["PProd","PProd_ORB_Part", "PProd_AST_Part", "PProd_FG_Part","qAST", "FG_Part", "ASP_Part", "FT_Part", "Team_Scoring_Poss", "Team_ORB%", "Team_Play%", "Team_ORB_Weight", "ORB_Part", "ScPoss", "FGxPoss", "FTxPoss", "TotPoss"], errors='ignore')




"playerID",
"year",
"stint",
"tmID",
"lgID",
"GP",
"GS",
"minutes",
"points",
"oRebounds",
"dRebounds",
"rebounds",
"assists",
"steals",
"blocks",
"turnovers",
"PF",
"fgAttempted",
"fgMade",
"ftAttempted",
"ftMade",
"threeAttempted",
"threeMade",
"dq",
"PostGP",
"PostGS",
"PostMinutes",
"PostPoints",
"PostoRebounds",
"PostdRebounds",
"PostRebounds",
"PostAssists",
"PostSteals",
"PostBlocks",
"PostTurnovers",
"PostPF",
"PostfgAttempted",
"PostfgMade",
"PostftAttempted",
"PostftMade",
"PostthreeAttempted",
"PostthreeMade",
"PostDQ"

Stops = Stops1 + Stops2
where:

Stops1 = STL + BLK * FMwt * (1 - 1.07 * DOR%) + DRB * (1 - FMwt)

FMwt (Forced Miss weight) = (DFG% * (1 - DOR%)) / (DFG% * (1 - DOR%) + (1 - DFG%) * DOR%)

DOR% = Opponent_ORB / (Opponent_ORB + Team_DRB)

DFG% = Opponent_FGM / Opponent_FGA teams["d_fgEf"] 

Stops2 = (((Opponent_FGA - Opponent_FGM - Team_BLK) / Team_MP) * FMwt * (1 - 1.07 * DOR%) + ((Opponent_TOV - Team_STL) / Team_MP)) * MP + (PF / Team_PF) * 0.4 * Opponent_FTA * (1 - (Opponent_FTM / Opponent_FTA))^2


Stop% = (Stops * Opponent_MP) / (Team_Possessions * MP)
With those numbers in hand, individual Defensive Rating can be computed:

DRtg = Team_Defensive_Rating + 0.2 * (100 * D_Pts_per_ScPoss * (1 - Stop%) - Team_Defensive_Rating)
where:

Team_Defensive_Rating = 100 * (Opponent_PTS / Team_Possessions)

D_Pts_per_ScPoss = Opponent_PTS / (Opponent_FGM + (1 - (1 - (Opponent_FTM / Opponent_FTA))^2) * Opponent_FTA*0.4)


In [ ]:
mPT["DOR%"] = mPT["d_oreb"] / (mPT["d_oreb"] + mPT["o_dreb"]) 

mPT["FMwt"] = (mPT["d_fgEf"] * (1 - mPT["DOR%"])) / (mPT["d_fgEf"]  * (1 - mPT["DOR%"] ) + (1 - mPT["d_fgEf"]) * mPT["DOR%"])

mPT["Stops1"] = mPT["steals"] + mPT["blocks"] * mPT["FMwt"] * (1 - 1.07 * mPT["DOR%"]) + mPT["dRebounds"] * (1 - mPT["FMwt"])

mPT["Stops2"]  = (((mPT["d_fga"] - mPT["d_fgm"] - mPT["o_blk"]) / mPT["minutes_team"]) * mPT["FMwt"] * (1 - 1.07 * mPT["DOR%"]) + ((mPT["d_to"] - mPT["o_stl"]) / mPT["minutes_team"])) * mPT["minutes"] + (mPT["PF"] / mPT["o_pf"] ) * 0.4 * mPT["d_fta"] * (1 - (mPT["d_ftm"] / mPT["d_fta"])) ** 2

mPT["Stops"] = mPT["Stops1"] + mPT["Stops2"]

mPT["Stop%"] = (mPT["Stops"]  * mPT["minutes_team"]) / (mPT["team_poss"] * mPT["minutes"])

mPT["D_Pts_per_ScPoss"] = mPT["d_pts"] / (mPT["d_fgm"] + (1 - (1 - (mPT["d_ftm"] / mPT["d_ftm"])) ** 2) * mPT["d_fta"]*0.4)

mPT["DRtg"] = mPT["def_rtg"] + 0.2 * (100 * mPT["D_Pts_per_ScPoss"] * (1 - mPT["Stop%"] ) - mPT["def_rtg"])

mPT = mPT.drop(columns =["DOR%", "FMwt", "Stops1", "Stops2", "Stops", "Stop%", "D_Pts_per_ScPoss"], errors='ignore')


mPT["per"] = (mPT["uper"] * (mPT["lg_pace"]/mPT["pace"])) * (15/mPT["lg_uper"])


mPT.to_csv('mPT.csv', index=False)